In [1]:
import pandas
import requests
import matplotlib.pyplot as plt

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC

# Klasifikace do více než dvou tříd
Jak algoritmy přistupují k úloze klasifikace do více tříd?
- Převedou úlohu s více třídami na problém dvou tříd, tedy binární klasifikaci (to dělá např. algoritmus SVM)
    - Varianta "Jeden proti všem (OvR)": Algoritmus vytvoří tolik klasifikátorů, kolik máme tříd, a trénuje každý jako binární klasifikaci jedné třidy oproti všem ostatním. Při predikci každý z klasifikátorů předpoví buď "svojí" třídu, nebo "ostatní třídy". Z těch, které vyberou "svojí" třídu, algoritmus pak vybere tu, která má nejvyšší skóre, nebo jistotu.
    - Jeden proti jednomu nebo každý proti každému (OvO): Algoritmus vytvoří tolik klasifikátorů, kolik je dvojic tříd. Při predikci pak algoritmus vybere tu třídu, která dostala nejvíc hlasů.
- Úlohu řeší přímo (to dělá např. algoritmus KNN nebo rozhodovací strom, angl. decision tree)

In [2]:
r = requests.get(
    "https://raw.githubusercontent.com/lutydlitatova/czechitas-datasets/main/datasets/soybean-1-spot.csv"
)
open("soybean-1-spot.csv", "wb").write(r.content)

26524

In [3]:
data = pandas.read_csv("soybean-1-spot.csv")
data

,leaf-shread,stem,lodging,stem-cankers,fruiting-bodies,external-decay,fruit-pods,fruit-spots,seed,seed-discolor,seed-size,shriveling,class
0,absent,norm,yes,absent,absent,absent,norm,absent,norm,absent,norm,absent,brown-spot
1,absent,norm,yes,absent,absent,absent,norm,absent,norm,absent,norm,absent,brown-spot
2,present,norm,yes,absent,absent,absent,norm,absent,norm,absent,norm,absent,brown-spot
3,present,norm,yes,absent,absent,absent,norm,absent,norm,absent,norm,absent,brown-spot
4,absent,norm,yes,absent,absent,absent,norm,absent,norm,absent,norm,absent,brown-spot
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,absent,abnorm,yes,above-sec-nde,absent,firm-and-dry,diseased,colored,norm,absent,norm,absent,frog-eye-leaf-spot
270,absent,abnorm,yes,above-sec-nde,absent,firm-and-dry,diseased,colored,norm,absent,norm,absent,frog-eye-leaf-spot
271,absent,abnorm,yes,above-sec-nde,absent,firm-and-dry,diseased,colored,norm,absent,norm,absent,frog-eye-leaf-spot
272,absent,abnorm,yes,above-sec-nde,absent,firm-and-dry,diseased,colored,norm,absent,norm,absent,frog-eye-leaf-spot


In [4]:
data["class"].value_counts()

brown-spot             92
alternarialeaf-spot    91
frog-eye-leaf-spot     91
Name: class, dtype: int64

In [5]:
data["stem"].value_counts()

norm      171
abnorm    103
Name: stem, dtype: int64

In [6]:
X = data.drop(columns=["class"])
y = data["class"]

Kategorické proměnné, v tomto případě všechny, musíme převést na číselné, pomocí metody one-hot (angl. one-hot encoding).

In [7]:
encoder = OneHotEncoder()
X = encoder.fit_transform(X)

In [10]:
pandas.DataFrame(X.toarray(), columns=encoder.get_feature_names_out()).head()

AttributeError: 'OneHotEncoder' object has no attribute 'get_feature_names_out'

Třídy převedeme na celočíselnou reprezentaci

In [12]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

Rozdělíme data na trénovací a testovací sadu.

Parametr stratify určuje, podle jakého sloupce chceme zachovat poměr hodnot. V našem případě chceme zachovat poměr tříd (aby v trénovacích i testovacích datech byly třídy podobně zastoupené).

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [14]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Jak vyhodnocujeme klasifikaci do více tříd?
Podíváme se, jak vypadá chybová matice pro více tříd. Funguje na stejném principu, jen je větší, a není na první pohled jasné, jak spočítat metriky jako precision nebo recall. Všimněme si ale, že základní metriky accuracy je stejná: Součet hodnot na diagonále (součet správně určených bodů) oproti velikosti datasetu.

In [16]:
confusion_matrix(y_test, y_pred)

array([[18,  0,  0],
       [ 4, 15,  0],
       [ 7,  1, 10]], dtype=int64)

In [17]:
ConfusionMatrixDisplay.from_estimator(
    clf,
    X_test,
    y_test,
    display_labels=encoder.classes_,
    cmap=plt.cm.Blues,
)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_estimator'

Metrikám, které pracují s dvěma typy chyb (False Positive, False Negative), můžeme dát parametr average s hodnotou "weighted". Spočítají tak vážený průměr přes všechny třídy. Například pro třídu alternarialeaf-spot bychom pracovali s hodnotami 18 (True Positive), 0 (False Positive) a 11 (False Negative).

In [19]:
print(round(accuracy_score(y_test, y_pred), 3))
print(round(f1_score(y_test, y_pred, average="weighted"), 3))

0.782
0.781


# Upravení více parametrů: Grid search
V minulé lekci jsme si ukazovali, že na trénování modelu může mít vliv hodnota jeho některého parametru. Například u algoritmu K Nearest Neighbors jsme zkoušeli nastavit různé hodnoty parametru n_neighbors pomocí for cyklu. Když bychom parametrů měli víc, můžeme použít vnořený for cyklus, ale brzy by se nám výsledky špatně porovnávaly. V knihovně scikit-learn existuje třída GridSearchCV, která nejlepší nastavení parametrů zjistí za nás.

Dělá to tak, že si trénovací data rozdělí na několik částí, a na těchto rozdělených datech trénuje a testuje model s různými parametry. Pak výsledky zprůměruje přes všechny díly a na základě toho určí nejlepší parametry.

In [20]:
model_1 = KNeighborsClassifier()
params_1 = {"n_neighbors": [1, 5, 7, 11, 13]}

clf_1 = GridSearchCV(model_1, params_1, scoring="f1_weighted")
clf_1.fit(X_train, y_train)

print(clf_1.best_params_)
print(round(clf_1.best_score_, 2))

{'n_neighbors': 11}
0.77


In [24]:
model_2 = LinearSVC(random_state=42)
params_2 = {"C": [0.01, 0.01, 0.1, 1]}

clf_2 = GridSearchCV(model_2, params_2, scoring="f1_weighted")
clf_2.fit(X_train, y_train)

print(clf_2.best_params_)
print(round(clf_2.best_score_, 2))

{'C': 1}
0.77


In [25]:
y_pred = clf.best_estimator_.predict(X_test)

AttributeError: 'KNeighborsClassifier' object has no attribute 'best_estimator_'

In [26]:
round(f1_score(y_test, y_pred, average="weighted"), 2)

0.78